# `transform.py`

 - what the transform is
  - has a basis and a grid, and evaluates that basis on its grid right? so it contains the transform matrices (call them transform matrices)? how are these called? but these are the matrices which, when multiplied against a vector of the coefficients of the spectral series, yields the series evaluated at the grid points

this file contains the `Transform` class. 

## `build()`

this method builds the transform matrices for each derivative order of the basis the transform requires (which is specified when the `Transform` object is initialized with the `derivs` argument).
Defining the transform matrix as $A_{(d\rho,d\theta,d\zeta)}$ for given derivatives of the basis ${(d\rho,d\theta,d\zeta)}$ (where each are integers), the matrix is used to transform a spectral basis with a given set of coefficients $\mathbf{c}$ into its values on the grid (given by `Transform.grid`) by 

$$ A\mathbf{c} = \mathbf{x}$$

where $\mathbf{x}$ is the values of the spectral basis evaluated at the grid points . 
$\mathbf{c}$ is a vector of length `Transform.basis.num_modes` (the number of modes in the basis), $\mathbf{x}$ is a vector of length `Transform.grid.num_nodes` (the number of nodes in the grid), and $A$ is a matrix of shape `(num_nodes,num_modes)`.


i.e. if a Fourier Series $f(\zeta) = 2 + 4*cos(\zeta)$, and the grid is $\zeta = (0,\pi)$, then $\mathbf{x} =\begin{bmatrix}0\\ \pi\end{bmatrix}$, $\mathbf{c}=\begin{bmatrix} 2\\ 4 \end{bmatrix}$, and  $A = \begin{bmatrix} 1 & cos(0)\\ 1& cos(\pi) \end{bmatrix} = \begin{bmatrix} 1& 1\\ 1& -1 \end{bmatrix} $ s.t. 
$$ A\mathbf{c} = \begin{bmatrix} 1& 1\\ 1& -1 \end{bmatrix} \begin{bmatrix} 2\\ 4 \end{bmatrix} = \begin{bmatrix} 6 \\ -2  \end{bmatrix}  $$


## `build_pinv()`

This function builds the pseudoinverse of the transform, which can then be used to take values of a given function that are given on `Transform.grid` and fit a spectral basis to them.
A couple different methods are available:

### `direct1`

With this method, 